In [ ]:
#Importar librerias
import datetime
import pandas as pd
import numpy as np
import seaborn
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#Importar bases de datos

listings = pd.read_csv(r"C:\Users\Javi\Desktop\proyectoR\AIRBNB MADRID\raw_data\listings.csv")
listings.head()

In [ ]:
reviews = pd.read_csv(r"C:\Users\Javi\Desktop\proyectoR\AIRBNB MADRID\raw_data\reviews.csv")
reviews.head()

In [ ]:
calendar = pd.read_csv(r"C:\Users\Javi\Desktop\proyectoR\AIRBNB MADRID\raw_data\calendar.csv")
calendar.head()

In [ ]:
#Cambiar tipos de datos para que podamos operar con ellos que nos interesan

In [ ]:
calendar.date.head()

In [ ]:
calendar.dtypes

In [ ]:
calendar["date"].dtype

In [ ]:
#De str a datetime
calendar["date"] = pd.to_datetime(calendar["date"])

In [ ]:
#Borrar signos que impiden hacer operaciones con ciertos numeros
def remove_sign(x, sign):
if type(x) is str:
x= float(x.replace(sign, "").replace(",", ""))
return x

In [ ]:
#borrar símbolo $ del precio y convertir en float
calendar.price = calendar.price.apply(remove_sign, sign="$")

In [ ]:
calendar.info()

In [ ]:
#Hacer lo mismo en listings
listings.info(verbose=True, null_counts=True)

In [ ]:
#De str a datetime
listings["host_since"] = pd.to_datetime(listings["host_since"])

In [ ]:
#QuitarSímbolo $ y convertir en float
listings["price"] = listings.price.apply(remove_sign, sign="$")

In [ ]:
#Quitar Símbolo % y convertir en float
listings["host_response_rate"] = listings["host_response_rate"].apply(remove_sign,sign ="%")

In [ ]:
listings.info(verbose=True)

In [ ]:
#Cambiar el formato de la fecha, del americano al formato europeo
#pd.to_datetime(calendar["date"].dt.strftime("%d-%m-%Y"))

In [ ]:
#Patrón de disponibilidad y precios

In [ ]:
Oferta de casas diaria
oferta_viviendas = calendar.groupby("date").apply(lambda x: x.notnull().sum())[["price"]]
#cambiar nombre columnas
oferta_viviendas = oferta_viviendas.rename(columns={"price":"Oferta disponible"})

In [ ]:
oferta_viviendas.head()

In [ ]:
#Precios medios
calendar_notnull= calendar[calendar.price.notnull()]
precio_medio_diario = calendar_notnull.groupby("date").mean()[["price"]]

In [ ]:
#cambiar nombre columnas
precio_medio_diario = precio_medio_diario.rename(columns ={"price":"Precio vivienda medio"})

In [ ]:
precio_medio_diario.head()

In [ ]:
#Grafico casas disponibles y precio medio
f, ax =plt.subplots(figsize=(15,6))
plt1 = seaborn.lineplot(x = oferta_viviendas.index, y= "Oferta disponible",
data = oferta_viviendas, color= "r", legend=False)
for tl in ax.get_yticklabels():
tl.set_color("r")

ax2= ax.twinx()
plt2= seaborn.lineplot(x = precio_medio_diario.index,
y= "Precio vivienda medio", data= precio_medio_diario, ax= ax2,
                       linestyle=":", legend=False)

In [ ]:
#Precio medio del alquiler en octubre de 2018
pmd_oct_18 = precio_medio_diario["2018-10-01":"2018-11-01"]
plot_pmd_oct_18= pmd_oct_18.plot(color="purple")
plot_pmd_oct_18.set_xlabel("Fecha")
plot_pmd_oct_18.set_ylabel("Precio")
plot_pmd_oct_18.legend(["Precio vivienda medio en Noviembre"],loc="upper right")
plt.show()

In [ ]:
#Precio medio del alquiler en diciembre de 2018
pmd_dic_18 = precio_medio_diario["2018-12-01":"2019-01-01"]
plot_pmd_dic_18=pmd_dic_18.plot(color="green")
plot_pmd_dic_18.set_xlabel("Fecha")
plot_pmd_dic_18.set_ylabel("Precio")
plot_pmd_dic_18.legend(["Precio vivienda medio en Diciembre"],loc="upper center")
plt.show()

In [ ]:
#Precio medio en una semana de Diciembre
pmd_dic_finde_2018 = precio_medio_diario["2018-12-01":"2018-12-20"]
plot_pmd_dic_finde_18 =pmd_dic_finde_2018.plot()
plot_pmd_dic_finde_18.xaxis.grid(True, which='minor',linestyle='-', linewidth=0.25)

In [ ]:
pmd_semanasanta_18 = precio_medio_diario["2018-03-20":"2018-04-05"]
plot_pmd_semanasanta_18 =pmd_semanasanta_18.plot()
plot_pmd_semanasanta_18.xaxis.grid(True, which='minor',linestyle='-', linewidth=0.25)

In [ ]:
# Tiempo del año en el que están disponible las viviendas

In [ ]:
tiempo_medio_disponible = calendar.groupby("listing_id").apply(lambda x: x.notnull().mean())[["price"]]
tiempo_medio_disponible = tiempo_medio_disponible.rename({"price":"available_ratio"}, axis="columns")

In [ ]:
#Viviendas disponibles menos de un 10% de todo el año
len(tiempo_medio_disponible[tiempo_medio_disponible.available_ratio < 0.10])

In [ ]:
len(tiempo_medio_disponible)

In [ ]:
#Viviendas disponibles más de un 90% de todo el año
len(tiempo_medio_disponible[tiempo_medio_disponible.available_ratio > 0.90])

In [ ]:
f, ax =plt.subplots(figsize=(15,6))
plot_tmd =seaborn.distplot(tiempo_medio_disponible, kde=False, color="red")
plot_tmd.set_xlabel("Porcentaje de tiempo al año disponible para alquilar", fontsize=15)
plot_tmd.set_ylabel("Cantidad de viviendas", fontsize=10)

In [ ]:
#Comprobar si a partir de septiembre existen cambios en la disponibildiad de las viviendas

In [ ]:
calendar_sept_dic = calendar.loc[(calendar["date"] >= "2018-01-17") & (calendar["date"]<= "2019-01-01")]

In [ ]:
tiempo_medio_disponible = calendar_sept_dic.groupby("listing_id").apply(lambda x: x.notnull().mean())[["price"]]
tiempo_medio_disponible = tiempo_medio_disponible.rename({"price":"available_ratio"}, axis="columns")

In [ ]:
f, ax =plt.subplots(figsize=(15,6))
ax =seaborn.distplot(tiempo_medio_disponible, kde=False)
ax.set_xlabel("% Vivienda en alquiler", fontsize=15)
ax.set_ylabel("Cantidad de viviendas", fontsize=10)

In [ ]:
len(tiempo_medio_disponible[tiempo_medio_disponible.available_ratio < 0.10])

In [ ]:
#Crear categorías para los precios
#Tendencia de los precios
calendar_notnull.price.head()

In [ ]:
listings.price.describe()

In [ ]:
#usar como precio bajo el primer cuartil (25%), usar como precio alto el tercer cuartil(75%)
def cat_precio(x, precio_bajo=35, precio_alto = 88):
if x<=precio_bajo:
    x="Precio Bajo"
elif x>= precio_alto:
    x="Precio Alto"
else:
    x="Precio Medio"
return x
listings["cat_precio"] = listings.price.apply(cat_precio)

In [ ]:
#Nueva columna con las 3 categorias de precios
precio_cal = listings[["id","price","review_scores_rating","number_of_reviews"]].dropna()
precio_cal.head()

In [ ]:
precio_cal.describe()

In [ ]:
#Relación precio y calificación
f, ax = plt.subplots(figsize=(15, 6))
seaborn.scatterplot(x='price',y='review_scores_rating',hue='number_of_reviews',alpha=0.5,data=precio_cal)

In [ ]:
#Quitamos outliers

In [ ]:
precio_cal_wo_outl = precio_cal

In [ ]:
def remove_outlier(df_in, col_name):
q1 = df_in[col_name].quantile(0.25)
q3 = df_in[col_name].quantile(0.75)
iqr = q3-q1 #Interquartile range
fence_low
= q1-1.5*iqr
fence_high = q3+1.5*iqr
df_out = df_in.loc[(df_in[col_name] > fence_low) & (df_in[col_name] < fence_high)]
return df_out

In [ ]:
precio_cal_wo_outl =remove_outlier(precio_cal_wo_outl, "price")

In [ ]:
f, ax = plt.subplots(figsize=(15, 6))
seaborn.scatterplot(x='price',y='review_scores_rating',hue='number_of_reviews',alpha=0.5,data=precio_cal_wo_outl)

In [ ]:
#Algunas de las variables que tienen un peso significativo en el precio

In [ ]:
# Quitar aquellos datos que no presenten ningún valor en el precio
listings_precios = listings[listings.price.notnull()]

In [ ]:
#Precios según barrio de Madrid
def plot_precio_barrio (colname, listings = listings, fig_row_size=11, fig_col_size=30
):
price_col=listings_precios.groupby(colname).mean()[["price"]]
price_col.reset_index(inplace=True)
f, ax = plt.subplots(figsize=(fig_row_size, fig_col_size))
seaborn.barplot(y = colname, x="price", data= price_col.sort_values(by="price", ascending=False))
ax.set_xlabel(xlabel="Precio")
ax.set_ylabel(ylabel="Barrio")

In [ ]:
plot_precio_barrio("neighbourhood_cleansed", listings = listings_precios)

In [ ]:
#Precio según vivenda/habitación alquilada
def plot_precio_tipov (colname, listings = listings, fig_row_size=11, fig_col_size=9):
price_col=listings_precios.groupby(colname).mean()[["price"]]
price_col.reset_index(inplace=True)
f, ax = plt.subplots(figsize=(fig_row_size, fig_col_size))
seaborn.barplot(y = colname, x="price", data= price_col.sort_values(by="price", ascending=False))
ax.set_xlabel(xlabel="Precio")
ax.set_ylabel(ylabel="Tipo de vivienda/habitación")
plot_precio_tipov("property_type", listings = listings_precios)

In [ ]:
#Precio según vivenda/habitación alquilada
def plot_valor_tipov (colname, listings = listings, fig_row_size=11, fig_col_size=30):
price_valor=listings_precios.groupby(colname).mean()[["review_scores_rating"]]
price_valor.reset_index(inplace=True)
f, ax = plt.subplots(figsize=(fig_row_size, fig_col_size))
seaborn.barplot(y = colname, x="review_scores_rating", data= price_valor.sort_values(by="review_scores_rating", ascending=False))
ax.set_xlabel(xlabel="Valoración de los usuarios")
ax.set_ylabel(ylabel="Barrio")
plot_valor_tipov("neighbourhood_cleansed", listings = listings_precios)

In [ ]:
#Relación entre el precio del alquiler y la velocidad de respuesta del host cuando reci be un mensaje
mod_listings_precios= listings_precios
mod_listings_precios["host_response_time"] = mod_listings_precios["host_response_time" ].replace({"a few days or more":"Un par de días o más",
"within a day":"Alrededor de un día",
"within an hour":"Alrededor de una hora",
"within a few hours":"En pocas horas"})

def plot_precio_respuesta (colname, listings = listings, fig_row_size=11, fig_col_size=
9):
price_col=listings_precios.groupby(colname).mean()[["price"]]
price_col.reset_index(inplace=True)
f, ax = plt.subplots(figsize=(fig_row_size, fig_col_size))
seaborn.barplot(y = colname, x="price", data= price_col.sort_values(by="price", ascending=False))
ax.set_xlabel(xlabel="Precio")
ax.set_ylabel(ylabel="Tiempo de respuesta")
plot_precio_respuesta("host_response_time", listings = listings_precios)

In [ ]:
#Buscar correlaciones numericas entre los valores de listings
listings_precios_num = listings_precios.select_dtypes(include=["float64","int"])
listings_precios.info()

In [ ]:
#quitar columnas irrelevantes
listings_precios_num = listings_precios.drop(["id","scrape_id",
    "host_id","latitude","longitude", "jurisdiction_names",
    "neighbourhood_group_cleansed","license","has_availability",
    "neighbourhood_group_cleansed","thumbnail_url","medium_url",
    "xl_picture_url","host_acceptance_rate"],axis=1)

In [ ]:
#Correlación
corr = listings_precios_num.corr()
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
cmap = seaborn.diverging_palette(220, 10, as_cmap=True)
f, ax = plt.subplots(figsize=(11, 9))
seaborn.heatmap(corr,mask=mask, cmap=cmap, vmax=.3, center=0,square=True, linewidths=.5, cbar_kws={"shrink": .5})